# Домашнее задание №1

На семинаре мы проходили метрики, необходимо реализовать некоторые из них

In [5]:
import numpy as np
import pandas as pd
import math

In [16]:
recommended_list = [143,156,1134,991,27,1543,3345,533,11,43]
prices_recommended = [400,60,40,90,60,340,70,190,110,240]

bought_list = [521,32,143,991]
prices_bought = [150,30,400,90]

## Задание 1. Реализовать метрики Recall@k и  Money Recall@k

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

$$\Large Recall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}}{|Rel_i|}$$

$\Large |Rel_i|$ -- количество релевантных товаров для пользователя $i$

$$\Large MoneyRecall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}\cdot Price(j)}{\sum_{s\in Rel_i}Price(s)}$$


In [13]:
def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list 
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)    
    recall = flags.sum() / len(bought_list)
    
    return recall



def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    prices_recommended = np.array(prices_recommended)
    prices_bought = np.array(prices_bought)
    
    bought_list = bought_list  
    recommended_list = recommended_list[:k]
    prices_recommended = prices_recommended[:k]
    prices_bought = prices_bought 
    
    flags = np.isin(recommended_list, bought_list)
    
    recall = np.dot(flags, prices_recommended) / prices_bought.sum()
            
    return recall

In [10]:
recall_at_k(recommended_list, bought_list, 5)

0.5

In [18]:
money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, 5)

0.7313432835820896

## Задание 2. Реализовать метрику MRR@k

Mean Reciprocal Rank

- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $\Large rank_j$
- Посчитать reciprocal rank = $\Large\frac{1}{rank_j}$

$$\Large  MMR(i)@k=\frac {1}{\min\limits_{j\in Rel(i)} rank_j}$$

In [37]:
def mrr_at_k(recommended_list, bought_list, k=1):
    recommended_list = np.array(recommended_list)
    bought_list = np.array(bought_list)
    
    amount_user = len(bought_list)
    rr = []
    for i in np.arange(amount_user):    
        relevant_indexes = np.nonzero(np.isin(recommended_list[i][:k], bought_list[i]))[0]
        if len(relevant_indexes) != 0:
            rr.append(1/(relevant_indexes[0]+1))
    
    if len(rr) == 0:
        return 0
    
    result = sum(rr)/amount_user
    
    return result

In [38]:
recommended_list_3_users = [[143,156,1134,991,27,1543,3345,533,11,43],
                            [1134,533,14,4,15,1543,1,99,27,3345],
                            [991,3345,27,533,43,143,1543,156,1134,11]]

bought_list_3_users= [[521,32,143], #user1
                      [143,156,991,43,11], #user2
                      [1,2]] #user3

In [39]:
mrr_at_k(recommended_list_3_users, bought_list_3_users, 5)

/tmp/ipykernel_4625/2398984129.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  bought_list = np.array(bought_list)


0.3333333333333333

## Задание 3*. Реализовать метрику nDCG@k
Normalized discounted cumulative gain. Эту метрику реализовать будет немного сложнее.

$$\Large DCG@K(i) = \sum_{j=1}^{K}\frac{\mathbb{1}_{r_{ij}}}{\log_2 (j+1)}$$


$\Large \mathbb{1}_{r_{ij}}$ -- индикаторная функция показывает что пользователь $i$ провзаимодействовал с продуктом $j$

Для подсчета $nDCG$ нам необходимо найти максимально возможный $DCG$ для пользователя $i$  и рекомендаций длины $K$.
Максимальный $DCG$ достигается когда мы порекомендовали максимально возможное количество релевантных продуктов и все они в начале списка рекомендаций.

$$\Large IDCG@K(i) = max(DCG@K(i)) = \sum_{j=1}^{K}\frac{\mathbb{1}_{j\le|Rel_i|}}{\log_2 (j+1)}$$

$$\Large nDCG@K(i) = \frac {DCG@K(i)}{IDCG@K(i)}$$

$\Large |Rel_i|$ -- количество релевантных продуктов для пользователя $i$



In [6]:
def ndcg_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list
    recommended_list = recommended_list[:k]
    
    flags = np.isin(recommended_list, bought_list)
    rank_list = []
    
    for i in np.arange(len(recommended_list)):
        if i < 2:
            rank_list.append(i+1)
        else:
            rank_list.append(math.log2(i+1))
    if len(recommended_list) == 0:
        return 0
    
    dcg = sum(np.divide(flags, rank_list)) / len(recommended_list)
    
    i_dcg = sum(np.divide(1, rank_list)) / len(recommended_list)
    
    result = dcg / i_dcg   
    
    return result

In [25]:
ndcg_at_k(recommended_list, bought_list, 5)

0.489938890671454